## Facebook denoiser

In [ ]:
!pip uninstall denoiser

In [ ]:
!pip install git+https://github.com/facebookresearch/denoiser#egg=denoiser

  Cloning https://github.com/facebookresearch/denoiser to /tmp/pip-install-j4uvw450/denoiser_a52e6581b97f4c109bce4d9f1b0aa9a3
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/denoiser /tmp/pip-install-j4uvw450/denoiser_a52e6581b97f4c109bce4d9f1b0aa9a3
  Resolved https://github.com/facebookresearch/denoiser to commit 8afd7c166699bb3c8b2d95b6dd706f71e1075df0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using 

In [ ]:
# /content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.m4a

In [ ]:
!ffmpeg -version

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [ ]:
import torch
import torchaudio
import subprocess
import os
from denoiser import pretrained

def convert_m4a_to_wav(input_path, output_path):
    subprocess.call(['ffmpeg', '-i', input_path, output_path])

def enhance_audio(input_path, output_path, segment_length=10):
    # m4a를 wav로 변환
    if input_path.endswith('.m4a'):
        wav_path = input_path.rsplit('.', 1)[0] + '.wav'
        convert_m4a_to_wav(input_path, wav_path)
        input_path = wav_path

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    model = pretrained.dns64().to(device)

    # 오디오 파일 로드
    noisy_audio, sr = torchaudio.load(input_path)

    # 모노로 변환 (스테레오인 경우)
    if noisy_audio.shape[0] > 1:
        noisy_audio = torch.mean(noisy_audio, dim=0, keepdim=True)

    # 모델의 샘플 레이트로 리샘플링
    if sr != model.sample_rate:
        noisy_audio = torchaudio.transforms.Resample(sr, model.sample_rate)(noisy_audio)

    # 세그먼트 길이 계산 (샘플 단위)
    segment_length_samples = segment_length * model.sample_rate

    enhanced_segments = []

    for start in range(0, noisy_audio.shape[1], segment_length_samples):
        end = min(start + segment_length_samples, noisy_audio.shape[1])
        segment = noisy_audio[:, start:end]

        # 패딩 추가
        if segment.shape[1] < segment_length_samples:
            padding = torch.zeros(1, segment_length_samples - segment.shape[1])
            segment = torch.cat([segment, padding], dim=1)

        segment = segment.to(device)

        with torch.no_grad():
            enhanced_segment = model(segment)

        # 패딩 제거
        if end == noisy_audio.shape[1]:
            enhanced_segment = enhanced_segment[:, :end-start]

        enhanced_segments.append(enhanced_segment.cpu())

    # 모든 세그먼트 연결
    enhanced = torch.cat(enhanced_segments, dim=1)

    # 차원 확인 및 조정
    print(f"Enhanced tensor shape before saving: {enhanced.shape}")
    if enhanced.dim() == 1:
        enhanced = enhanced.unsqueeze(0)
    elif enhanced.dim() > 2:
        enhanced = enhanced.squeeze()
        if enhanced.dim() > 2:
            raise ValueError(f"Unexpected tensor dimension: {enhanced.dim()}")



    # wav로 저장
    torchaudio.save(output_path, enhanced, model.sample_rate)
    print(f"Enhanced audio saved to {output_path}")

    # 임시 wav 파일 삭제
    if input_path.endswith('.wav') and input_path != output_path:
        os.remove(input_path)


# 사용 예
input_path = "/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.m4a"
output_path = "/content/drive/MyDrive/IAP/STT/enhanced_audio.wav"
enhance_audio(input_path, output_path)


Using device: cuda
Enhanced tensor shape before saving: torch.Size([1, 590, 160000])


RuntimeError: Failed to create input filter: "time_base=1/16000:sample_rate=16000:sample_fmt=flt:channel_layout=0x0" (Invalid argument)
Exception raised from add_src at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/filter_graph.cpp:91 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7be8981f6897 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x7be8981a6b25 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #2: torio::io::FilterGraph::add_src(AVFilter const*, std::string const&) + 0x37f (0x7be889ea471f in /usr/local/lib/python3.10/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: torio::io::FilterGraph::add_audio_src(AVSampleFormat, AVRational, int, unsigned long) + 0x31 (0x7be889ea4821 in /usr/local/lib/python3.10/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #4: torio::io::get_audio

In [ ]:
import torch
import torchaudio
import subprocess
import os
from denoiser import pretrained

def convert_m4a_to_wav(input_path, output_path):
    subprocess.call(['ffmpeg', '-i', input_path, output_path])

def enhance_audio(input_path, output_path, segment_length=10):
    # m4a를 wav로 변환
    if input_path.endswith('.m4a'):
        wav_path = input_path.rsplit('.', 1)[0] + '.wav'
        convert_m4a_to_wav(input_path, wav_path)
        input_path = wav_path

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    model = pretrained.dns64().to(device)

    # 오디오 파일 로드
    noisy_audio, sr = torchaudio.load(input_path)

    # 모노로 변환 (스테레오인 경우)
    if noisy_audio.shape[0] > 1:
        noisy_audio = torch.mean(noisy_audio, dim=0, keepdim=True)

    # 모델의 샘플 레이트로 리샘플링
    if sr != model.sample_rate:
        noisy_audio = torchaudio.transforms.Resample(sr, model.sample_rate)(noisy_audio)

    # 세그먼트 길이 계산 (샘플 단위)
    segment_length_samples = segment_length * model.sample_rate

    enhanced_segments = []

    for start in range(0, noisy_audio.shape[1], segment_length_samples):
        end = min(start + segment_length_samples, noisy_audio.shape[1])
        segment = noisy_audio[:, start:end]

        # 패딩 추가
        if segment.shape[1] < segment_length_samples:
            padding = torch.zeros(1, segment_length_samples - segment.shape[1])
            segment = torch.cat([segment, padding], dim=1)

        segment = segment.to(device)

        with torch.no_grad():
            enhanced_segment = model(segment)

        # 패딩 제거
        if end == noisy_audio.shape[1]:
            enhanced_segment = enhanced_segment[:, :end-start]

        enhanced_segments.append(enhanced_segment.cpu())

    # 모든 세그먼트 연결
    enhanced = torch.cat(enhanced_segments, dim=1)

    # 차원 확인 및 조정
    if enhanced.dim() == 1:
        enhanced = enhanced.unsqueeze(0)
    elif enhanced.dim() > 2:
        enhanced = enhanced.squeeze()
        if enhanced.dim() > 2:
            raise ValueError(f"Unexpected tensor dimension: {enhanced.dim()}")

    torchaudio.save(output_path, enhanced, model.sample_rate)
    print(f"Enhanced audio saved to {output_path}")

    # 임시 wav 파일 삭제
    if input_path.endswith('.wav') and input_path != output_path:
        os.remove(input_path)

# 사용 예
# input_path = "noisy_audio.m4a"
# output_path = "enhanced_audio.wav"
enhance_audio(input_path, output_path)


Using device: cuda


RuntimeError: Failed to create input filter: "time_base=1/16000:sample_rate=16000:sample_fmt=flt:channel_layout=0x0" (Invalid argument)
Exception raised from add_src at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/filter_graph.cpp:91 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7be8981f6897 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x7be8981a6b25 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #2: torio::io::FilterGraph::add_src(AVFilter const*, std::string const&) + 0x37f (0x7be889ea471f in /usr/local/lib/python3.10/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: torio::io::FilterGraph::add_audio_src(AVSampleFormat, AVRational, int, unsigned long) + 0x31 (0x7be889ea4821 in /usr/local/lib/python3.10/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #4: torio::io::get_audio

## 커스텀 인공지능

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.wav
<br>
/content/drive/MyDrive/IAP/STT/enhance.wav

In [ ]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import os

class AudioEnhancementModel(nn.Module):
    def __init__(self):
        super(AudioEnhancementModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(32, 1, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.conv4(x)
        return x

def improve_audio_quality_gpu(input_file, output_file, model, device, batch_size=8192, overlap=1024):
    # 오디오 파일 로드
    waveform, sample_rate = torchaudio.load(input_file)

    # 모노로 변환 (스테레오인 경우)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # 배치 처리를 위한 준비
    total_samples = waveform.shape[1]
    enhanced_waveform = torch.zeros_like(waveform)

    for start in range(0, total_samples, batch_size - overlap):
        end = min(start + batch_size, total_samples)

        # 현재 배치 추출
        batch = waveform[:, start:end]

        # GPU로 데이터 이동
        batch = batch.to(device)

        # 모델 통과
        with torch.no_grad():
            enhanced_batch = model(batch)

        # 결과를 CPU로 이동
        enhanced_batch = enhanced_batch.cpu()

        # 오버랩 부분 처리
        if start == 0:
            enhanced_waveform[:, start:end] = enhanced_batch
        else:
            # 이전 배치와 현재 배치의 오버랩 부분을 블렌딩
            blend = torch.linspace(0, 1, overlap)
            enhanced_waveform[:, start:start+overlap] *= (1 - blend)
            enhanced_waveform[:, start:start+overlap] += enhanced_batch[:, :overlap] * blend
            enhanced_waveform[:, start+overlap:end] = enhanced_batch[:, overlap:]

    # 향상된 오디오 저장
    torchaudio.save(output_file, enhanced_waveform, sample_rate)

def detect_poor_quality_gpu(waveform, sample_rate, threshold_snr=15, threshold_volume=-30, batch_size=8192):
    total_samples = waveform.shape[1]
    snr_sum = 0
    volume_sum = 0

    for start in range(0, total_samples, batch_size):
        end = min(start + batch_size, total_samples)
        batch = waveform[:, start:end]

        # SNR 계산
        signal_power = torch.mean(batch**2)
        noise = batch - torchaudio.functional.lowpass_biquad(batch, sample_rate, 200)
        noise_power = torch.mean(noise**2)
        snr = 10 * torch.log10(signal_power / noise_power)

        # 볼륨 레벨 계산 (dB)
        volume_db = 20 * torch.log10(torch.mean(torch.abs(batch)))

        snr_sum += snr.item()
        volume_sum += volume_db.item()

    # 평균 SNR과 볼륨 계산
    avg_snr = snr_sum / (total_samples // batch_size + (1 if total_samples % batch_size else 0))
    avg_volume = volume_sum / (total_samples // batch_size + (1 if total_samples % batch_size else 0))

    # 품질 판단
    is_poor_quality = (avg_snr < threshold_snr) or (avg_volume < threshold_volume)

    return is_poor_quality, avg_snr, avg_volume

# 모델 가중치 저장 함수 (학습 후 사용)
def save_model_weights(model, save_path):
    torch.save(model.state_dict(), save_path)
    print(f"Model weights saved to {save_path}")

# 모델 가중치 로드 함수
def load_model_weights(model, load_path):
    if os.path.exists(load_path):
        model.load_state_dict(torch.load(load_path))
        print(f"Model weights loaded from {load_path}")
    else:
        print(f"No weights file found at {load_path}. Using untrained model.")

# 메인 실행 코드
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # 모델 인스턴스 생성
    model = AudioEnhancementModel().to(device)

    # 가중치 파일 경로 설정
    weights_path = "audio_enhancement_model_weights.pth"

    # 가중치 로드
    load_model_weights(model, weights_path)

    # 입력 및 출력 파일 경로 설정
    input_file = "/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.wav"
    output_file = "/content/drive/MyDrive/IAP/STT/enhance.wav"

    # 음질 향상
    improve_audio_quality_gpu(input_file, output_file, model, device)

    # 음질 감지
    waveform, sample_rate = torchaudio.load(output_file)
    is_poor, snr, volume = detect_poor_quality_gpu(waveform, sample_rate)

    print(f"Poor quality: {is_poor}")
    print(f"Average SNR: {snr:.2f} dB")
    print(f"Average Volume: {volume:.2f} dB")

    # 참고: 모델 학습 후 가중치를 저장하려면 아래 줄의 주석을 해제하세요
    # save_model_weights(model, weights_path)

Using device: cuda
No weights file found at audio_enhancement_model_weights.pth. Using untrained model.
Poor quality: True
Average SNR: 23.79 dB
Average Volume: -33.48 dB


## CPU Version

/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.wav
<br>
/content/drive/MyDrive/IAP/STT/enhance_cpu.wav

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [ ]:
import numpy as np
import librosa
import soundfile as sf
from scipy import signal

def improve_audio_quality(input_file, output_file, target_sr=22050):
    # 오디오 파일 로드
    y, sr = librosa.load(input_file, sr=None)

    # 리샘플링
    if sr != target_sr:
        y = librosa.resample(y, sr, target_sr)

    # 노이즈 제거 (간단한 고주파 필터)
    b, a = signal.butter(10, 2000/(target_sr/2), btype='lowpass')
    y_filtered = signal.filtfilt(b, a, y)

    # 볼륨 정규화
    y_normalized = librosa.util.normalize(y_filtered)

    # 향상된 오디오 저장
    sf.write(output_file, y_normalized, target_sr)

    return y_normalized, target_sr

def detect_poor_quality(audio, sr, threshold_snr=15, threshold_volume=-30):
    # SNR (Signal-to-Noise Ratio) 계산
    signal_power = np.mean(audio**2)
    noise = audio - librosa.effects.percussive(audio)
    noise_power = np.mean(noise**2)
    snr = 10 * np.log10(signal_power / noise_power)

    # 볼륨 레벨 계산 (dB)
    volume_db = 20 * np.log10(np.mean(np.abs(audio)))

    # 품질 판단
    is_poor_quality = snr < threshold_snr or volume_db < threshold_volume

    return is_poor_quality, snr, volume_db

# 사용 예시
input_file = "/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.wav"
output_file = "/content/drive/MyDrive/IAP/STT/enhance_cpu.wav"

# 음질 향상
improved_audio, sr = improve_audio_quality(input_file, output_file)

# 음질 감지
is_poor, snr, volume = detect_poor_quality(improved_audio, sr)

print(f"Poor quality: {is_poor}")
print(f"SNR: {snr:.2f} dB")
print(f"Volume: {volume:.2f} dB")

TypeError: resample() takes 1 positional argument but 3 were given

## Demucs

In [ ]:
!pip install demucs

In [ ]:
!pip uninstall omegaconf
!pip install omegaconf

Found existing installation: omegaconf 1.4.1
Uninstalling omegaconf-1.4.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/omegaconf-1.4.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/omegaconf/*
Proceed (Y/n)? Y
  Successfully uninstalled omegaconf-1.4.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=604d5260abe684c5a2557efaa645303cf12f921de4e68d221e91faf8b4410813
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hydra-core 

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.wav
<br>
/content/drive/MyDrive/IAP/STT/enhance_demucs.wav

In [ ]:
import torch
import torchaudio
import numpy as np
from demucs import pretrained
from demucs.apply import apply_model

def enhance_audio(input_file, output_file):
    # GPU 사용 가능 여부 확인
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # Demucs 모델 로드 (여기서는 'demucs' 모델을 사용)
    model = pretrained.get_model('demucs')

    # 오디오 파일 로드
    wav, sr = torchaudio.load(input_file)
    wav = wav.to(device)

    # 스테레오를 모노로 변환 (필요한 경우)
    if wav.shape[0] > 1:
        wav = wav.mean(0, keepdim=True)

    # 모델 적용
    ref = wav.mean(0)
    wav = (wav - ref.mean()) / ref.std()
    sources = apply_model(model, wav.unsqueeze(0), device=device, split=True, overlap=0.25)
    sources = sources * ref.std() + ref.mean()

    # 향상된 오디오 추출 (여기서는 'vocals' 트랙을 사용)
    enhanced = sources[0, 0]  # 첫 번째 배치, vocals 트랙

    # 결과를 CPU로 이동 및 numpy 배열로 변환
    enhanced = enhanced.cpu().numpy()

    # 클리핑 방지
    enhanced = np.clip(enhanced, -1, 1)

    # 향상된 오디오 저장
    torchaudio.save(output_file, torch.from_numpy(enhanced).unsqueeze(0), sr)

    print(f"Enhanced audio saved to {output_file}")

# 사용 예
input_file = "/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.wav"  # 입력 오디오 파일 경로
output_file = "/content/drive/MyDrive/IAP/STT/enhance_demucs.wav"  # 출력 오디오 파일 경로
enhance_audio(input_file, output_file)


## NVIDIA NeMO

### Install Dependencies

In [4]:
# Install dependencies
!pip install nemo_toolkit[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 50.4

### Import Library

In [3]:
import torch
from nemo.collections.asr.models import EncDecCTCModel
from nemo.collections.asr.parts.preprocessing import AudioSegment

### Set Device

In [7]:
# Check if CUDA is available and set the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Download pre-trained model

In [8]:
model = EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")
model = model.to(device)  # Move the model to the appropriate device

[NeMo I 2024-07-21 10:26:08 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.23.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2024-07-21 10:26:08 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.23.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2024-07-21 10:26:08 common:924] Instantiating model from pre-trained checkpoint
[NeMo I 2024-07-21 10:26:09 features:289] PADDING: 16
[NeMo I 2024-07-21 10:26:10 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.23.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


### Function

In [9]:
def enhance_audio_nemo(input_file, output_file):
    # Load audio file
    audio = AudioSegment.from_file(input_file)

    # Convert audio to features and move to the appropriate device
    features = model.preprocessor(
        input_signal=torch.tensor(audio.samples, device=device).unsqueeze(0),
        length=torch.tensor([audio.num_samples], device=device)
    )

    # Apply the model for enhancement
    with torch.no_grad():
        enhanced_features, _ = model.encoder(audio_signal=features)

    # Convert enhanced features back to audio
    enhanced_audio = model.decoder(encoder_output=enhanced_features)

    # Move the enhanced audio back to CPU for saving
    enhanced_audio = enhanced_audio.squeeze().cpu().numpy()
    AudioSegment(enhanced_audio, sample_rate=audio.sample_rate).write(output_file)

### Usage

In [10]:
# Usage
input_file = "/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.wav"
output_file = "/content/drive/MyDrive/IAP/STT/enhance_sample.wav"
enhance_audio_nemo(input_file, output_file)

print(f"Enhanced audio saved to {output_file}")

TypeError: Number of input arguments provided (1) is not as expected. Function has 2 total inputs with 2 mandatory inputs.

In [11]:
# Install NeMo
!pip install nemo_toolkit[asr]

import torch
from nemo.collections.asr.models import EncDecCTCModel
from nemo.collections.asr.parts.preprocessing import AudioSegment

# Check if CUDA is available and set the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Download the pre-trained Conformer-CTC model for ASR
# This model can be used for speech enhancement as a side effect
model = EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")
model = model.to(device)  # Move the model to the appropriate device

def enhance_audio_nemo(input_file, output_file):
    # Load audio file
    audio = AudioSegment.from_file(input_file)

    # Convert audio to features and move to the appropriate device
    features, features_length = model.preprocessor(
        input_signal=torch.tensor(audio.samples, device=device).unsqueeze(0),
        length=torch.tensor([audio.num_samples], device=device)
    )

    # Apply the model for enhancement
    with torch.no_grad():
        enhanced_features, _ = model.encoder(audio_signal=features, length=features_length)

    # Convert enhanced features back to audio
    enhanced_audio, _ = model.decoder(encoder_output=enhanced_features)

    # Move the enhanced audio back to CPU for saving
    enhanced_audio = enhanced_audio.squeeze().cpu().numpy()
    AudioSegment(enhanced_audio, sample_rate=audio.sample_rate).write(output_file)

# Usage
input_file = "/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.wav"
output_file = "/content/drive/MyDrive/IAP/STT/enhance_sample.wav"
enhance_audio_nemo(input_file, output_file)

print(f"Enhanced audio saved to {output_file}")

[NeMo I 2024-07-21 10:28:09 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.23.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2024-07-21 10:28:09 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.23.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2024-07-21 10:28:09 common:924] Instantiating model from pre-trained checkpoint
[NeMo I 2024-07-21 10:28:10 features:289] PADDING: 16
[NeMo I 2024-07-21 10:28:11 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.23.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


ValueError: not enough values to unpack (expected 2, got 1)

## Libsora

In [15]:
!pip install noisereduce

In [16]:
import numpy as np
import librosa
import soundfile as sf
import noisereduce as nr
from scipy import signal

def enhance_audio(input_file, output_file, sr=22050):
    # 오디오 파일 로드
    y, sr = librosa.load(input_file, sr=sr)

    # 노이즈 제거
    y_denoised = nr.reduce_noise(y=y, sr=sr)

    # 음질 향상
    # 1. 고주파 부스트 (프리엠퍼시스)
    y_eq = librosa.effects.preemphasis(y_denoised)

    # 2. 음량 정규화
    y_normalized = librosa.util.normalize(y_eq)

    # 음량 증폭
    target_dB = -20  # 목표 음압 레벨 (dB)
    rms = np.sqrt(np.mean(y_normalized**2))
    scalar = 10**(target_dB / 20) / rms
    y_amplified = y_normalized * scalar

    # 음성 강조 필터링 (대역 통과 필터)
    low_cut = 300.0  # 저주파 컷오프 (Hz)
    high_cut = 3000.0  # 고주파 컷오프 (Hz)
    nyquist = 0.5 * sr
    low = low_cut / nyquist
    high = high_cut / nyquist
    b, a = signal.butter(N=6, Wn=[low, high], btype='band')
    y_filtered = signal.filtfilt(b, a, y_amplified)

    # 후처리: 잔여 노이즈 제거 및 평활화
    y_final = nr.reduce_noise(y=y_filtered, sr=sr)

    # 결과 저장
    sf.write(output_file, y_final, sr)


# 사용 예
input_file = "/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.wav"
output_file = "/content/drive/MyDrive/IAP/STT/enhance_sample.wav"
enhance_audio(input_file, output_file)


In [17]:
## 처음 버전 -> 이거 우선 점지
import numpy as np
import librosa
import soundfile as sf
from scipy import signal

def enhance_audio(input_file, output_file, sr=22050):
    # 오디오 파일 로드
    y, sr = librosa.load(input_file, sr=sr)

    # 노이즈 제거
    # 1. 스펙트로그램 계산
    D = librosa.stft(y)
    D_mag, D_phase = librosa.magphase(D)

    # 2. 노이즈 프로파일 추정 (첫 몇 프레임을 노이즈로 가정)
    noise_profile = np.mean(D_mag[:, :10], axis=1, keepdims=True)

    # 3. 스펙트럼 감산
    D_mag_denoised = np.maximum(D_mag - noise_profile, 0)

    # 4. 역변환
    y_denoised = librosa.istft(D_mag_denoised * D_phase)

    # 음질 향상
    # 1. 고주파 부스트 (프리엠퍼시스)
    y_eq = librosa.effects.preemphasis(y_denoised)

    # 2. 음량 정규화
    y_normalized = librosa.util.normalize(y_eq)

    # 음량 증폭
    target_dB = -20  # 목표 음압 레벨 (dB)
    rms = np.sqrt(np.mean(y_normalized**2))
    scalar = 10**(target_dB / 20) / rms
    y_amplified = y_normalized * scalar

    # 음성 강조 필터링 (대역 통과 필터)
    low_cut = 300.0  # 저주파 컷오프 (Hz)
    high_cut = 3000.0  # 고주파 컷오프 (Hz)
    nyquist = 0.5 * sr
    low = low_cut / nyquist
    high = high_cut / nyquist
    b, a = signal.butter(N=6, Wn=[low, high], btype='band')
    y_filtered = signal.filtfilt(b, a, y_amplified)

    # 결과 저장
    sf.write(output_file, y_filtered, sr)

# 사용 예
input_file = "/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.wav"
output_file = "/content/drive/MyDrive/IAP/STT/enhance_sample_V1.wav"
enhance_audio(input_file, output_file)



In [19]:
## 최초 버전
import numpy as np
import librosa
import soundfile as sf

def enhance_audio(input_file, output_file, sr=22050):
    # 오디오 파일 로드
    y, sr = librosa.load(input_file, sr=sr)

    # 노이즈 제거
    # 1. 스펙트로그램 계산
    D = librosa.stft(y)
    D_mag, D_phase = librosa.magphase(D)

    # 2. 노이즈 프로파일 추정 (첫 몇 프레임을 노이즈로 가정)
    noise_profile = np.mean(D_mag[:, :10], axis=1, keepdims=True)

    # 3. 스펙트럼 감산
    D_mag_denoised = np.maximum(D_mag - noise_profile, 0)

    # 4. 역변환
    y_denoised = librosa.istft(D_mag_denoised * D_phase)

    # 음질 향상
    # 1. 고주파 부스트
    y_eq = librosa.effects.preemphasis(y_denoised)

    # 2. 음량 정규화
    y_normalized = librosa.util.normalize(y_eq)

    # 결과 저장
    sf.write(output_file, y_normalized, sr)

# 사용 예
input_file = "/content/drive/MyDrive/IAP/STT/인공지능플랫폼_16-02.wav"
output_file = "/content/drive/MyDrive/IAP/STT/enhance_sample_V0.wav"
enhance_audio(input_file, output_file)
